In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
#df = pd.read_csv(r"C:\Users\HP\Downloads\medicinal_plants_single_disease.csv")
df = pd.read_csv(r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv")




In [13]:
df['Content'] = df['Disease Cured'] + " " + df['Medicinal Benefits']


In [14]:
df = df.dropna(subset=['Content'])


In [15]:
df['Content'] = df['Content'].fillna('')


In [16]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Content'])

In [23]:

def load_data():
    file_path =r"C:\Users\HP\Downloads\Medicinal Plant identification\Medicinal Plant identification\medicinal_plants_single_disease.csv"




  # Update with actual path
    df = pd.read_csv(file_path)
    return df

def train_model(df):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['Disease Cured'])
    return vectorizer, tfidf_matrix

def recommend_plant(disease_query, df, vectorizer, tfidf_matrix):
    query_tfidf = vectorizer.transform([disease_query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    best_match_idx = cosine_similarities.argmax()  # Get the best match
    best_match = df.iloc[best_match_idx]
    
    print(f"Best recommendation for '{disease_query}':\n")
    print(f"Plant Name: {best_match['Plant Name']}")
    print(f"Disease Cured: {best_match['Disease Cured']}")
    print(f"Benefits: {best_match['Medicinal Benefits']}")
    print(f"Similarity Score: {cosine_similarities[best_match_idx]:.2f}")
    print("-" * 40)

if __name__ == "__main__":
    df = load_data()
    vectorizer, tfidf_matrix = train_model(df)
    disease_query = input("Enter the disease or symptom for recommendation: ")
    recommend_plant(disease_query, df, vectorizer, tfidf_matrix)



Enter the disease or symptom for recommendation: oral infection
Best recommendation for 'oral infection':

Plant Name: Betel
Disease Cured:  Oral infections
Benefits: Antiseptic properties, improves digestion, relieves congestion.
Similarity Score: 0.77
----------------------------------------


In [25]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Prepare dataset for supervised learning, creating labels based on plants
df['Label'] = df['Plant Name'].astype('category').cat.codes
X_train, X_test, y_train, y_test = train_test_split(df['Disease Cured'], df['Label'], test_size=0.2, random_state=42)

In [26]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = vectorizer.transform(X_test).toarray()

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train_tfidf.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(df['Label'].unique()), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_tfidf, y_train, epochs=10, validation_data=(X_test_tfidf, y_test))

# Save the trained model to .h5 format
model.save("plant_recommendation_model.h5")



Epoch 1/10


1/1 [==============================] - 3s 3s/step - loss: 3.5126 - accuracy: 0.0385 - val_loss: 3.5341 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 111ms/step - loss: 3.4974 - accuracy: 0.0385 - val_loss: 3.5396 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 124ms/step - loss: 3.4822 - accuracy: 0.1154 - val_loss: 3.5452 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 129ms/step - loss: 3.4672 - accuracy: 0.1538 - val_loss: 3.5508 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 119ms/step - loss: 3.4522 - accuracy: 0.1538 - val_loss: 3.5564 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 127ms/step - loss: 3.4372 - accuracy: 0.1538 - val_loss: 3.5622 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 122ms/step - loss: 3.4223 - accuracy: 0.1923 - val_loss: 3.5679 - val_accuracy: 0.

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
# Load the model
model = tf.keras.models.load_model("plant_recommendation_model.h5")

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("plant_recommendation_model.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpmqv94yss\assets


INFO:tensorflow:Assets written to: C:\Users\HP\AppData\Local\Temp\tmpmqv94yss\assets
